In [1]:
import pandas as pd
with pd.HDFStore("store.h5") as store:
    ddf = store['ddf']

In [6]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from tqdm import tnrange
embed = hub.Module(".\sentence_wise_email\module\module_useT")

tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    
#    a = np.empty((0,512))
#    for i in tnrange(0,len(ddf),1000, desc="Processing Titles"):
#        a = np.vstack((a,session.run(embed(ddf['title'][i:i+1000].tolist()))))  
        
#    with pd.HDFStore("store.h5") as store:
#        store['ddf-use-title'] = pd.DataFrame(a,index=ddf.index)
        
    a = np.empty((0,512))
    for i in tnrange(0,len(ddf),1000, desc="Processing Body"):
        a = np.vstack((a,session.run(embed(ddf['selftext'][i:i+1000].tolist()))))
        
    with pd.HDFStore("store.h5") as store:
        store['ddf-use-selftext'] = pd.DataFrame(a,index=ddf.index)

C:\Users\Jamie\Anaconda3\lib\site-packages\tables\path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'ddf-use-selftext'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


HDF5ExtError: Problems creating the Array.

Index(['created_utc', 'edited', 'gilded', 'isEdited', 'label', 'locked',
       'm_esh', 'm_info', 'm_nah', 'm_nta', 'm_yta', 'num_comments', 'over_18',
       'score', 'selftext', 'spoiler', 'sum', 't_esh', 't_info', 't_nah',
       't_nta', 't_yta', 'title', 'upvote_ratio'],
      dtype='object')

In [ ]:
#Export for pretraining
from tqdm import tqdm
from nltk.tokenize import sent_tokenize
import re
from random import random

def x(match):
    ta = "the asshole" if random() > 0.025 else "TA"
    if(match.string.lower() == "aita"):
        return "Am I" + ta
    return "Would I be " + ta

with open('corpus.txt','w',encoding='utf-8') as f:
    for idx, row in tqdm(ddf.iterrows(), desc="Exporting Dataset", total=len(ddf)):
        lines = re.sub(r"\b(wib|ai)ta\b",x,row['title'],flags=re.IGNORECASE).split("\n")
        lines += re.sub(r"\b(wib|ai)ta\b",x,row['selftext'],flags=re.IGNORECASE).split("\n")
        sentences = []
        for line in lines:
            sentences += sent_tokenize(line)
        sentences = [i.strip() for i in sentences if len(i.strip()) > 2]
        if len(sentences) > 1:
            f.write('\n'.join(sentences) + '\n\n')

In [ ]:
#quick experiment with tokenization - apparently nltk's built in works better than training on corpus
import nltk.tokenize.punkt
import pickle
import re
tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()

corpus = []
#with open("sentencetokenizer.pk","wb") as f:
for i, row in df.iterrows():
    corpus.append(row['title'])
    corpus.append(row['selftext'])
st = ' '.join(corpus)
print(len(st.replace("&#x200B;","")))
print(len(st.replace("&#x200B;\n","\n")))
print(len(re.sub(r"&#x200B;\n?","",st)))
    #pickle.dump(tokenizer.train('\n'.join(corpus)),f)
from nltk.tokenize import sent_tokenize
#sent_tokenize = tokenizer.tokenize
smp = df.sample(10000)
with open("training_a.txt",'w', encoding="utf-8") as f:
    for idx, row in smp.iterrows():
         f.write('\n'.join(tokenizer.tokenize(row['title'])) + '\n')
         #f.writelines
         f.write('\n'.join(tokenizer.tokenize(re.sub(r"&#x200B;\n?","",row['selftext']))))
         f.write('\n-------------------------------------------\n')
with open("training_b.txt",'w', encoding="utf-8") as f:
    for idx, row in smp.iterrows():
         f.write('\n'.join(sent_tokenize(row['title'])) + '\n')
         #f.writelines
         f.write('\n'.join(sent_tokenize(row['selftext'])))
         f.write('\n-------------------------------------------\n')

In [ ]:
def proc(row):
    out = row['title'] + '\n' + row['selftext']
ddf['text'] = ddf.apply(proc,axis=1)
train, test = train_test_split(ddf[['text','a_op','a_to']], test_size=0.5)
val, test = train_test_split(test, test_size=0.5)
train.to_csv('train.csv')
val.to_csv('val.csv')
test.to_csv('test.csv')